# Calculation of historical performances of all flights

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
% pylab inline
import operator
import re
from collections import OrderedDict
import seaborn as sns
import pickle
import os.path
from matplotlib.colors import ListedColormap

Populating the interactive namespace from numpy and matplotlib


//anaconda/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
pd.set_option('display.max_columns', None)

## Loading the flight data

In [3]:
df = pd.read_csv("../Data/FlightData/OnTime_OnPerformance_Dec2014-Mar2017.csv")

//anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest
0,0,2014,4,12,24,3,2014-12-24,US,20355,US,N655AW,1852,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37,North Carolina,36,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,530,531.0,1.0,1.0,0.0,0.0,0001-0559,18.0,549.0,634.0,5.0,631,639.0,8.0,8.0,0.0,0.0,0600-0659,0.0,NaN,0.0,61.0,68.0,45.0,1.0,130.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,1,2014,4,12,24,3,2014-12-24,US,20355,US,N524UW,1853,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,745,748.0,3.0,3.0,0.0,0.0,0700-0759,17.0,805.0,948.0,6.0,1014,954.0,-20.0,0.0,0.0,-2.0,1000-1059,0.0,NaN,0.0,269.0,246.0,223.0,1.0,1587.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,2,2014,4,12,24,3,2014-12-24,US,20355,US,N524UW,1853,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,12889,1288903,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,1135,1132.0,-3.0,0.0,0.0,-1.0,1100-1159,20.0,1152.0,1136.0,7.0,1150,1143.0,-7.0,0.0,0.0,-1.0,1100-1159,0.0,NaN,0.0,75.0,71.0,44.0,1.0,255.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,3,2014,4,12,24,3,2014-12-24,US,20355,US,N915US,1854,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,1555,1605.0,10.0,10.0,0.0,0.0,1500-1559,17.0,1622.0,2141.0,6.0,2144,2147.0,3.0,3.0,0.0,0.0,2100-2159,0.0,NaN,0.0,229.0,222.0,199.0,1.0,1773.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,4,2014,4,12,24,3,2014-12-24,US,20355,US,N578UW,1854,14908,1490803,32575,SNA,"Santa Ana, CA",CA,6,California,91,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,1240,1250.0,10.0,10.0,0.0,0.0,1200-1259,11.0,1301.0,1449.0,8.0,1452,1457.0,5.0,5.0,0.0,0.0,1400-1459,0.0,NaN,0.0,72.0,67.0,48.0,1.0,338.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [5]:
df.shape

(13263051, 67)

## Setting index for the data frame


The index setting takes a lot of computational time, but later on it speeds up the calculation by more than an order of magnitude when we filter and sort our data frame.

In [6]:
dfind = df.set_index(["Carrier", "Origin", "Dest"]).sort_index()

## Changing formats of date columns

In [7]:
df["FlightDate"] = pd.to_datetime(df["FlightDate"])
dfind["FlightDate"] = pd.to_datetime(dfind["FlightDate"])

## Function that returns True or False, based on whether two times are close to each other

In [8]:
def check_time_vicinity(t1, t2, tlimit):
    # t1, t2 are the two times that we want to compare
    # tlimit: It is used to set a window around the CRS departure time, and get departure time window
    # Returns: True or False
    t1 = datetime.datetime.strptime(str(t1).zfill(4), "%H%M")
    t2 = datetime.datetime.strptime(str(t2).zfill(4), "%H%M")
    difference = abs(t1 - t2)
    minutes = difference.total_seconds() / 60
    # Since we are deaeling with 2400 time format, there is a periodicity and we take care of that 
    # using the following IFTTT
    if (minutes >= 720):
        minutes_corrected = 1440 - minutes
    else:
        minutes_corrected = minutes
    if (minutes_corrected > tlimit):
        return False
    else:
        return True

## Function to calculate aggregated history for a given row

In [9]:
def get_history(row, thisDate, ndays, dfrow, thisCRSDep, time_limit):
    # row: given row (flight)
    # thisDate: the flight date for the given row (flight)
    # ndays: Number of days that we want to calculate the history for, for given row (flight)
    # dfrow: the data frame obtained after filtering the whole data based on carrier, origin and departure airports
    # thisCRSDep: the departure time for a given row (flight)
    # time_limit: It is used to set a window around the CRS departure time, and get departure time window
    # Return: List of aggregated historical results
    
    # Get the date before ndays of the current flight (given row)
    DateBeforeNdays = thisDate - pd.DateOffset(ndays)
    
    # Get the data frame only for the last ndays data
    dfrowNdays = dfrow[(dfrow.FlightDate >= DateBeforeNdays - pd.DateOffset(1)) 
                   & (dfrow.FlightDate < (thisDate - pd.DateOffset(1)))]
    try:
        # Filter out the data frame to contain only those records for which the departure time window matches
        dfrowNdaysFinal = dfrowNdays[dfrowNdays.CRSDepTime.apply(check_time_vicinity, 
                                                             t2=thisCRSDep, tlimit=time_limit)].reset_index()

        # Aggregated properties:
        # Number of flights
        Nflights = len(dfrowNdaysFinal)
        # Median departure delay (minutes)
        DepDelayMedian = dfrowNdaysFinal.DepDelay.median()
        # Maximum departure delay (minutes)
        DepDelayMax = dfrowNdaysFinal.DepDelay.max()
        # Median arrival delay (minutes)
        ArrDelayMedian = dfrowNdaysFinal.ArrDelay.median()
        # Maximum arrival delay (minutes)
        ArrDelayMax = dfrowNdaysFinal.ArrDelay.max()
        # Median taxi out time (minutes)
        TaxiOutMedian = dfrowNdaysFinal.TaxiOut.median()
        # Median taxi in time (minutes)
        TaxiInMedian = dfrowNdaysFinal.TaxiIn.median()
        # Number of cancelled flights
        Ncan = sum(dfrowNdaysFinal.Cancelled == 1)
        # Number of diversions
        Ndiv = sum(dfrowNdaysFinal.DivAirportLandings)
    # Sometimes we get AttributeError when we do not find any flight after filtering the data frames.
    # We simply ignore that exception, and store 0 and NaN for these 9 variables.
    except AttributeError:
        Nflights = 0
        DepDelayMedian = np.nan
        DepDelayMax = np.nan
        ArrDelayMedian = np.nan
        ArrDelayMax = np.nan
        TaxiOutMedian = np.nan
        TaxiInMedian = np.nan
        Ncan = np.nan
        Ndiv = np.nan
    # Returning the list containing these 9 results
    return(list((Nflights, DepDelayMedian, DepDelayMax, ArrDelayMedian, ArrDelayMax, TaxiOutMedian, 
      TaxiInMedian, Ncan, Ndiv)))

## Actual calculation now happens here...


Note that the following loop takes huge amount of CPU time. In my system (iMac, 4 GHz Intel Core i7, 16 GB 1867 MHz DDR3), it took about 4 days! So, think twice before running the following code. Ofcourse, one can choose to loop over a smaller number of rows to run this faster. Also, note that we calculate the history for three values of ndays: 10, 20 and 30. 

In [10]:
%%time
# Creating list of list for nday=10
hist10 = []
# Creating list of list for nday=20
hist20 = []
# Creating list of list for nday=30
hist30 = []
# Looping over all rows in the data frame
# Note below that "current flight" refers to the flight for a given row (the current row in the loop)
for row in np.arange(len(df)):
    # get date of current flight
    thisDate = df.FlightDate[row]
    # get CRS departure time of current flight
    thisCRSDep = df.CRSDepTime[row]
    thisCRSDepFormatted = datetime.datetime.strptime(str(thisCRSDep).zfill(4), "%H%M")
    # get hour of the current flight
    thisHour = thisCRSDepFormatted.hour
    # get minute of the current flight
    thisMin = thisCRSDepFormatted.minute
    # gupdate the current flight's date with hour and minute
    thisDate = thisDate + pd.DateOffset(hour = thisHour, minute = thisMin)
    # filter the data frame by using carrier, departure and origin airports
    dfrow = dfind.loc[df.Carrier[row]].loc[df.Origin[row]].loc[df.Dest[row]]
    # append the list of list which get results by calling get_history() function
    hist10.append(get_history(row, thisDate, 10, dfrow, thisCRSDep, time_limit=60))
    hist20.append(get_history(row, thisDate, 20, dfrow, thisCRSDep, time_limit=60))
    hist30.append(get_history(row, thisDate, 30, dfrow, thisCRSDep, time_limit=60))
    if (row % 100000 == 0):
            print("Data processed for ", row, " rows")



Data processed for  0  rows
Data processed for  100000  rows
Data processed for  200000  rows
Data processed for  300000  rows
Data processed for  400000  rows
Data processed for  500000  rows
Data processed for  600000  rows
Data processed for  700000  rows
Data processed for  800000  rows
Data processed for  900000  rows
Data processed for  1000000  rows
Data processed for  1100000  rows
Data processed for  1200000  rows
Data processed for  1300000  rows
Data processed for  1400000  rows
Data processed for  1500000  rows
Data processed for  1600000  rows
Data processed for  1700000  rows
Data processed for  1800000  rows
Data processed for  1900000  rows
Data processed for  2000000  rows
Data processed for  2100000  rows
Data processed for  2200000  rows
Data processed for  2300000  rows
Data processed for  2400000  rows
Data processed for  2500000  rows
Data processed for  2600000  rows
Data processed for  2700000  rows
Data processed for  2800000  rows
Data processed for  2900000  

## Concatenate all the historical results with original data flight data

In [11]:
histCols = ["Nflights", "DepDelayMedian", "DepDelayMax", "ArrDelayMedian", "ArrDelayMax", "TaxiOutMedian", 
      "TaxiInMedian", "Ncan", "Ndiv"]

dfh10 = pd.DataFrame(hist10, columns=list(pd.Series(histCols).map(lambda x: x+"_10")))
dfh20 = pd.DataFrame(hist20, columns=list(pd.Series(histCols).map(lambda x: x+"_20")))
dfh30 = pd.DataFrame(hist30, columns=list(pd.Series(histCols).map(lambda x: x+"_30")))
df1 = pd.concat([df, dfh10, dfh20, dfh30], axis=1)

## Save the resulted data frame to csv

In [12]:
df1.to_csv("../Data/FlightData/OnTime_OnPerformance_Dec2014-Mar2017_WithHistory_Window60mins.csv")

In [13]:
df1.head()

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,Nflights_10,DepDelayMedian_10,DepDelayMax_10,ArrDelayMedian_10,ArrDelayMax_10,TaxiOutMedian_10,TaxiInMedian_10,Ncan_10,Ndiv_10,Nflights_20,DepDelayMedian_20,DepDelayMax_20,ArrDelayMedian_20,ArrDelayMax_20,TaxiOutMedian_20,TaxiInMedian_20,Ncan_20,Ndiv_20,Nflights_30,DepDelayMedian_30,DepDelayMax_30,ArrDelayMedian_30,ArrDelayMax_30,TaxiOutMedian_30,TaxiInMedian_30,Ncan_30,Ndiv_30
0,0,2014,4,12,24,3,2014-12-24,US,20355,US,N655AW,1852,14492,1449202,34492,RDU,"Raleigh/Durham, NC",NC,37,North Carolina,36,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,530,531.0,1.0,1.0,0.0,0.0,0001-0559,18.0,549.0,634.0,5.0,631,639.0,8.0,8.0,0.0,0.0,0600-0659,0.0,NaN,0.0,61.0,68.0,45.0,1.0,130.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,9,-1.0,7.0,-7.0,9.0,14.0,6.0,0.0,0.0,16,-3.0,7.0,-10.0,9.0,12.0,5.5,0.0,0.0,19,-3.0,21.0,-11.0,15.0,13.0,5.0,0.0,0.0
1,1,2014,4,12,24,3,2014-12-24,US,20355,US,N524UW,1853,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,745,748.0,3.0,3.0,0.0,0.0,0700-0759,17.0,805.0,948.0,6.0,1014,954.0,-20.0,0.0,0.0,-2.0,1000-1059,0.0,NaN,0.0,269.0,246.0,223.0,1.0,1587.0,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,10,-2.0,40.0,-6.0,29.0,13.5,19.0,0.0,0.0,20,-4.5,40.0,-8.0,29.0,14.5,21.0,0.0,0.0,23,-5.0,40.0,-8.0,29.0,14.0,19.0,0.0,0.0
2,2,2014,4,12,24,3,2014-12-24,US,20355,US,N524UW,1853,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,12889,1288903,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,1135,1132.0,-3.0,0.0,0.0,-1.0,1100-1159,20.0,1152.0,1136.0,7.0,1150,1143.0,-7.0,0.0,0.0,-1.0,1100-1159,0.0,NaN,0.0,75.0,71.0,44.0,1.0,255.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,20,-5.0,7.0,-4.5,12.0,18.5,6.5,0.0,0.0,38,-5.0,158.0,-4.5,172.0,19.0,6.0,0.0,0.0,44,-4.5,158.0,-4.5,172.0,18.5,6.0,0.0,0.0
3,3,2014,4,12,24,3,2014-12-24,US,20355,US,N915US,1854,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,1555,1605.0,10.0,10.0,0.0,0.0,1500-1559,17.0,1622.0,2141.0,6.0,2144,2147.0,3.0,3.0,0.0,0.0,2100-2159,0.0,NaN,0.0,229.0,222.0,199.0,1.0,1773.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,20,-2.0,22.0,-3.5,31.0,14.5,9.5,0.0,0.0,36,-1.0,114.0,-2.0,126.0,15.5,10.0,0.0,0.0,41,-1.0,114.0,-3.0,126.0,16.0,10.0,0.0,0.0
4,4,2014,4,12,24,3,2014-12-24,US,20355,US,N578UW,1854,14908,1490803,32575,SNA,"Santa Ana, CA",CA,6,California,91,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,1240,1250.0,10.0,10.0,0.0,0.0,1200-1259,11.0,1301.0,1449.0,8.0,1452,1457.0,5.0,5.0,0.0,0.0,1400-1459,0.0,NaN,0.0,72.0,67.0,48.0,1.0,338.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,10,-0.5,26.0,4.0,28.0,14.0,7.5,0.0,0.0,20,-1.0,26.0,3.5,28.0,14.5,7.0,0.0,0.0,23,0.0,26.0,6.0,28.0,15.0,7.0,0.0,0.0
